In [15]:
from CWScopeTesting import *
from WPI_SCA_LIBRARY.FileFormat import *

# init test scope object
scope = CWScopeTesting()

(ChipWhisperer Other ERROR|File util.py:362) Setting unknown attribute offset in <class 'chipwhisperer.capture.scopes.OpenADC.OpenADC'>


Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 5951 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 5951 bytes


In [12]:
# Captures a single power trace and plots it
scope.capture_one_trace()

In [17]:
# Capture 100 traces and store to hdf5 
scope.cw_scope.cw_to_hdf5("test_file.hdf5", "test_experiment", 100, True, False)

In [30]:
# read in the stored trace data from the HDF5 file
with h5py.File('test_file.hdf5', "a") as data_file:
    fileClass = HDF5FileClass('test_file.hdf5', fileInputType="existing", data_file=data_file)
    exp = fileClass.experiments['test_experiment']
       
    traces = exp.dataset["traces"]
    print(traces.readData(0))
   

[ 0.02441406 -0.19628906 -0.13867188 ... -0.05566406  0.02050781
  0.03417969]
